In [1]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()

In [2]:
import pandas as pd
import numpy as np
import os
import sys
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import tqdm

In [191]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import cv2

import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from keras.optimizers import Adam

In [192]:
# Graph
from py2neo import Graph

# 1. Input data

In [4]:
PATH_DATA = "/Users/david/TFM_DATA/spec"
data_files = os.listdir(PATH_DATA)

In [5]:
# PATH triplets
PATH_TRIPLETS = os.path.join("..","triplets", "triplets.csv")
df = pd.read_csv(PATH_TRIPLETS, delimiter= ";")

In [6]:
# List
triplets_input = list(df["output"])
size_triplets_input = len(triplets_input)

# Sample
triplets_input_v1 = triplets_input[:1000]
size_triplets_input = len(triplets_input_v1)

# Artists
artists_labels = np.sort(df.a1.unique())
df_artists = pd.DataFrame(data = artists_labels, columns = ["artist"]).reset_index()
df_artists.columns = ["id","artist"]

# DF Artists with artists as index
df_artists_index = df_artists.set_index("artist")
num_artists = df_artists.shape[0]

### Catalan music

In [193]:
graph = Graph(bolt = True, host = "localhost", name = "Spotify", user = "neo4j", password = "qrks")

In [194]:
query = """
        MATCH (g:Genre)-[:GEN_ART]->(a:Artist)
        MATCH (a:Artist)-[:REL_ART]->(a2:Artist)
        WHERE toLower(g.genre_id) =~ '.*catala.*'
        RETURN a.artist_id, a2.artist_id

"""

In [195]:
cursor = graph.run(query)
dfneo = pd.DataFrame.from_records(cursor, columns=cursor.keys())

In [197]:
set_art_cat = set()
for ii, row in dfneo.iterrows():
    set_art_cat.add(row["a.artist_id"])
    set_art_cat.add(row["a2.artist_id"])

In [200]:
#TODO: select couple of artists
sel_art = list(set_art_cat)

In [217]:
df_artists_cat = df_artists[df_artists.artist.isin(set_art_cat)]["artist"].reset_index()

In [222]:
df_artists_cat["id"] = df_artists_cat.index
df_artists_cat.drop('index', axis=1, inplace = True)

## Labels 

In [224]:
df_artists_gen = df_artists_cat.copy()

In [225]:
#labels_mat = keras.utils.to_categorical(df_artists_index.id)
labels_mat = keras.utils.to_categorical(df_artists_gen.id)

In [227]:
labels = dict()
for i, row in df_artists_gen.iterrows():
    labels[row.artist] = labels_mat[row.id,].astype(int)

In [278]:
num_artists = len(labels.keys())

## Songs

In [233]:
df_gen = df[df.a1.isin(set_art_cat)]

## 1.1 Dataframe Classiset_art_cat

In [306]:
df1 = df_gen[["a1","tr1","win1","ini1","fin1"]].copy()
df2 = df_gen[["a1","tr2","win2","ini2","fin2"]].copy()

# Column names
colnames = ["art","tr","win","ini","fin"]
df1.columns = colnames
df2.columns = colnames

In [307]:
df_concat = pd.concat([df1, df2])
df_concat.drop_duplicates(inplace = True)

In [308]:
df_concat["tr"] = df_concat.tr + "__" + df_concat.win.astype(str) +  \
    "__" + df_concat.ini.astype(str) + "__" + df_concat.fin.astype(str) + ".jpg"

In [309]:
df_concat.tail()

art                                       tr  \
13869032  0InCPtI0kadS7s3cZrcbbY    7hiceWoDGGP4RadNyGxRdm__2__40__70.jpg   
13869034  0InCPtI0kadS7s3cZrcbbY   7hiceWoDGGP4RadNyGxRdm__4__80__110.jpg   
13869041  0InCPtI0kadS7s3cZrcbbY  7hiceWoDGGP4RadNyGxRdm__7__140__170.jpg   
13869051  0InCPtI0kadS7s3cZrcbbY  7hiceWoDGGP4RadNyGxRdm__6__120__150.jpg   
13869061  0InCPtI0kadS7s3cZrcbbY  7hiceWoDGGP4RadNyGxRdm__9__180__210.jpg   

          win  ini  fin  
13869032    2   40   70  
13869034    4   80  110  
13869041    7  140  170  
13869051    6  120  150  
13869061    9  180  210

In [310]:
df_concat = df_concat.reset_index().drop("index", axis = 1)

## 1.2 Data Generator

In [312]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, scaling, x_col, y_col=None, batch_size=10, num_classes=None, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.x_col = x_col
        self.y_col = y_col
        self.dim = (int(x_col / scaling), int(y_col / scaling), 1) #one input channel
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        idx = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in idx]
        
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        pass

    def __get_data(self, batch):
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        
        # Get the list of image files and corresponding artists
        df_imgs_files = self.df.iloc[batch]
        
        list_imgs = list(df_imgs_files["tr"])
        list_art = list(df_imgs_files["art"])
        
        for ii in range(len(list_imgs)):
            
            # Read image using cv2
            path_img = os.path.join(PATH_DATA, list_imgs[ii])
            img = cv2.cvtColor(cv2.imread(path_img), cv2.COLOR_BGR2GRAY)
            img = np.round(img / 255.,5)
            img = cv2.resize(img, (self.y_col, self.x_col))
            img = cv2.resize(img, (self.dim[1], self.dim[0]))
            img = np.expand_dims(img, axis = 2) # add the dimension of the channel 
            
            # Put it into X matrix
            X[ii,] = img
            y[ii] = labels[list_art[ii]]

        return X, y

In [313]:
x_col, y_col = (256, 937)
batch_size = 5
num_classes = df_artists.shape[0]
shuffle = False
scaling = 1

training_generator = DataGenerator(df=df_concat, 
                                   scaling = scaling,
                                   x_col=x_col, 
                                   y_col=y_col,
                                   batch_size=batch_size, 
                                   num_classes=num_artists,
                                   shuffle=False)

# Model

In [315]:
def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x


def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters

def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AveragePooling2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters

def dense_net(filters, growth_rate, classes, dense_block_size, layers_in_block, in_shape):
    input_img = Input(shape=in_shape)
    x = Conv2D(3, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)

    dense_x = BatchNormalization()(x)
    dense_x = Activation('relu')(x)

    dense_x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(dense_x)
    for block in range(dense_block_size - 1):
        dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
        dense_x, filters = transition_block(dense_x, filters)

    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x = BatchNormalization()(dense_x)
    dense_x = Activation('relu')(dense_x)
    dense_x = GlobalAveragePooling2D()(dense_x)

    output = Dense(classes, activation='softmax')(dense_x)

    return Model(input_img, output)

## Quick model

In [316]:
def quick_CNN(input_shape, num_classes):

    model = Sequential()
    model.add(Conv2D(2, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(4, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

## Instantiate the model

In [320]:
dense_block_size = 4
layers_in_block = 2

in_shape = (int(x_col / scaling), int(y_col / scaling), 1)
growth_rate = 2
classes = num_artists

#model = dense_net(growth_rate * 2, growth_rate, classes, dense_block_size, layers_in_block, in_shape)
model = quick_CNN(input_shape=in_shape, num_classes = classes)

## Parameters

In [321]:
# training
epochs = 10
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [322]:
model.fit_generator(generator = training_generator, 
          epochs=epochs, 
          shuffle=True)

Epoch 1/10
383/383 [==============================] - 73s 190ms/step - loss: 3.7707 - acc: 0.0512
Epoch 2/10
383/383 [==============================] - 66s 172ms/step - loss: 2.8787 - acc: 0.0799
Epoch 3/10
383/383 [==============================] - 66s 173ms/step - loss: 2.8723 - acc: 0.0799
Epoch 4/10
383/383 [==============================] - 71s 186ms/step - loss: 2.8675 - acc: 0.0799
Epoch 5/10
383/383 [==============================] - 39s 103ms/step - loss: 2.8639 - acc: 0.0799
Epoch 6/10
383/383 [==============================] - 29s 76ms/step - loss: 2.8608 - acc: 0.0799
Epoch 7/10
383/383 [==============================] - 29s 76ms/step - loss: 2.8585 - acc: 0.0799
Epoch 8/10
383/383 [==============================] - 29s 76ms/step - loss: 2.8565 - acc: 0.0799
Epoch 9/10
383/383 [==============================] - 29s 76ms/step - loss: 2.8548 - acc: 0.0799
Epoch 10/10
383/383 [==============================] - 29s 76ms/step - loss: 2.8535 - acc: 0.0799
